In [ ]:
import os
import sys
import time

import numpy as np
import pygame
from gymnasium import Env
from gymnasium.spaces import Discrete, Dict, Box

from Agents.agent import Agent
# from Agents.fov_points import get_fov_points
from Agents.overlap_detection import detect_overlapping_points
from Constants.constants import WHITE, RED, BLUE, SCREEN_WIDTH, SCREEN_HEIGHT, WALLS, FOV_RADIUS
from Walls.collision_detection import detect_collision
from Walls.wall_class import Walls

In [ ]:
class GameEnv(Env):
    def __init__(self, render_mode='human'):
        super(GameEnv, self).__init__()

        # defining the screen dimension for render purpose
        self.screen_width = SCREEN_WIDTH
        self.screen_height = SCREEN_HEIGHT
        self.render_mode = render_mode

        # defining the observation and action spaces for all the agents
        self.observation_space = Dict({
            'predator_position': Box(low=np.array([0, 0], dtype=np.float32),
                                     high=np.array([SCREEN_WIDTH, SCREEN_HEIGHT], dtype=np.float32),
                                     dtype=np.float32),
            'vision': Dict({
                'vision_points': Discrete(2)
            }),
        })

        # defining the action space based on total number of predator and prey
        # since we are training only one agent so, defining only the necessary number of actions
        self.action_space = Discrete(5)
        # 5 for rotate
        # clockwise, anti-clock
        # move front, move back and wait

        self.total_steps = 0

        self.number_of_predator = 1

        self.predator_agent = None

        self.predator_total_reward = 0

        self.obs = None

        # start the tick timer
        self.start_time = 0
        self.total_running_time = 10

        # the pygame window should be initialized in the render function
        # initializing the pygame
        pygame.init()

        # setting the screen size
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption('Multi Agent Environment(simple)')

        # initializing the font
        pygame.font.init()
        self.font = pygame.font.Font(None, 18)

        # for the wall initializations
        self.wall = Walls(pygame)
        self.walls = None

    def agent_init(self):
        predator_agents = Agent('predator', 0)

        self.predator_agent = predator_agents

    def _get_obs(self):
        observation = {
            'predator_position': self.predator_agent.current_position,
            'predator_angle': self.predator_agent.angle,
            'vision': detect_overlapping_points(self.predator_agent.current_position, WALLS),
        }

        return observation

    def _max_right(self):
        max_right = 0

        for wall in self.walls:
            if wall.right > max_right:
                max_right = wall.right
        return max_right

    # the usual reset function
    def reset(self, seed=0):
        self.start_time = time.time()

        self.agent_init()
        self.wall.clear_walls()
        self.walls = self.wall.make_wall(WALLS)

        self.total_steps = 0
        self.predator_total_reward = 0

        predator = self.predator_agent

        # for predator in self.predator_agents:
        predator.agent_reset(width=self.screen_width, height=self.screen_height, walls=self.walls)
        # observation.append([predator.index, predator.agent, predator.current_position])

        # setting the predator and prey to their initial position

        self.predator_agent = predator


        # all the variable values inside the observation space needs to be sent inside the observation variable
        # for this level purpose we decided to add the dictionary observation
        # set the observation to a dictionary
        observation = self._get_obs()
        self.obs = observation

        return observation, seed

    def step(self, action):
        # initializing the return variables
        done = False
        reward = 0
        truncated = False
        info = {}
        current_time = time.time()

        elapsed_time = current_time - self.start_time
        # handles the pygame window event when closing
        # !if the window still crashes pygame.event needs to be managed properly
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                pygame.quit()
        self.predator_agent.step_update(action, range_x=self.screen_width, range_y=self.screen_height)
        self.predator_agent = detect_collision(self.predator_agent, self.walls)

        # observation needs to be set a dictionary

        self.total_steps += 1
        print(self._max_right())
        # for wall in self.walls:
        if self.predator_agent.current_position[0] > self._max_right():
            reward += 100
            done = True

        if elapsed_time >= self.total_running_time:
            done = True
        """
        here lies the most important task
        handling the rewards
        """
        reward += 0.01
        self.render()

        # it will update the total reward every step
        observation = self._get_obs()
        self.predator_total_reward = reward
        self.obs = observation

        return observation, reward, done, truncated, info

    def render(self):
        if self.render_mode == 'human':
            screen = self.screen

            screen.fill(WHITE)
            predator = self.predator_agent
            pygame.draw.circle(screen, RED, predator.center, predator.radius)
            pygame.draw.line(screen, RED, predator.center, predator.draw_direction_end, 5)

            for key, wall in WALLS.items():
                pygame.draw.rect(screen, BLUE, (wall['x'], wall['y'], wall['width'], wall['height']))

            pygame.display.update()

    def close(self):
        pygame.quit()

In [ ]:
env = GameEnv()

In [ ]:
from stable_baselines3 import PPO

In [ ]:
log_path = os.path.join('Training', 'Logs', 'Level_01_PPO')
baseline_path = os.path.join('Training', 'Models', 'Level_01_PPO')

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)
env.reset()
model.learn(total_timesteps=500000)

In [ ]:
env.close()

In [ ]:
import numpy as np

FOV_RADIUS = 100  # Adjust this value according to your requirements
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600

def get_fov_points(agent_position):
    fov_points = {}
    x, y = agent_position
    # Loop that iterates from 0 to 359, representing all possible angles in degrees.
    for angle in range(360):
        # Convert angle to radians
        radians = np.deg2rad(angle)
        # Calculate new coordinates within FOV based on polar coordinates 
        x_coord = int(x + FOV_RADIUS * np.cos(radians))
        y_coord = int(y - FOV_RADIUS * np.sin(radians))  # Adjust y-axis as needed
        # Check if the calculated x_coord and y_coord fall within the screen boundaries.
        if 0 <= x_coord < SCREEN_WIDTH and 0 <= y_coord < SCREEN_HEIGHT:
            # If yes, then add the coordinates to the fov_points dictionary
            fov_points[(x_coord, y_coord)] = 0

    # Print the axis
    x_axis = [(x, y) for x in range(0, SCREEN_WIDTH + 1)]
    y_axis = [(x, y) for y in range(0, SCREEN_HEIGHT + 1)]
    
    for point in x_axis:
        fov_points[point] = 'x'
    for point in y_axis:
        fov_points[point] = 'y'

    return fov_points

# Example usage:
agent_position = (400, 300)  # Example agent position
fov = get_fov_points(agent_position)
for coord, value in fov.items():
    print(f"Coordinate: {coord}, Value: {value}")


In [14]:
import pygame
import numpy as np

FOV_RADIUS = 100  # Adjust this value according to your requirements
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
AGENT_COLOR = (255, 0, 0)  # Red
POINT_COLOR = (0, 255, 0)  # Green
BACKGROUND_COLOR = (255, 255, 255)  # White

def get_fov_points(agent_position):
    fov_points = {}
    x, y = agent_position
    for angle in range(360):
        radians = np.deg2rad(angle)
        x_coord = int(x + FOV_RADIUS * np.cos(radians))
        y_coord = int(y - FOV_RADIUS * np.sin(radians))
        if 0 <= x_coord < SCREEN_WIDTH and 0 <= y_coord < SCREEN_HEIGHT:
            fov_points[(x_coord, y_coord)] = 0

    x_axis = [(x, y) for x in range(0, SCREEN_WIDTH + 1)]
    y_axis = [(x, y) for y in range(0, SCREEN_HEIGHT + 1)]

    for point in x_axis:
        fov_points[point] = 'x'
    for point in y_axis:
        fov_points[point] = 'y'

    return fov_points

def draw_fov(fov_points, agent_position):
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("FOV Visualization")

    running = True
    clock = pygame.time.Clock()

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        keys = pygame.key.get_pressed()
        x, y = agent_position

        if keys[pygame.K_UP]:
            y -= 1
        if keys[pygame.K_DOWN]:
            y += 1
        if keys[pygame.K_LEFT]:
            x -= 1
        if keys[pygame.K_RIGHT]:
            x += 1

        agent_position = (x, y)

        screen.fill(BACKGROUND_COLOR)  # Set the background color to white

        for coord, value in fov_points.items():
            if value == 'x':
                pygame.draw.circle(screen, POINT_COLOR, coord, 2)
            elif value == 'y':
                pygame.draw.circle(screen, POINT_COLOR, coord, 2)
            else:
                pygame.draw.circle(screen, POINT_COLOR, coord, 2)

        pygame.draw.circle(screen, AGENT_COLOR, agent_position, 5)

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

# Example usage:
agent_position = (400, 300)  # Example agent position
fov = get_fov_points(agent_position)
draw_fov(fov, agent_position)
